<a href="https://colab.research.google.com/github/ajraymond27/D.ai-TRIP-MEDIA/blob/master/Deep_Dream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title Hit This GPU Bruh
#Check if GPU is connected
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# How much GPU, tho?
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# What about RAM?
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#@title Art Vandalay, Importer Exporter
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import IPython.display as display
import PIL.Image
from tensorflow.keras.preprocessing import image
import cv2
import os
import time

In [ ]:
#@title Download, Deprocess, and Show
# Download an image and read it into a NumPy array.
def download(path, max_dim=None):
  img = PIL.Image.open(path)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)

# Normalize an image
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

# Display an image
def show(img):
  display.display(PIL.Image.fromarray(np.array(img)))

In [ ]:
#@title Prepare the feature extraction model
#@markdown Download and prepare a pre-trained image classification model. You will use InceptionV3 which is similar to the model originally used in DeepDream. Note that any pre-trained model will work, although you will have to adjust the layer names below if you change this.

#@markdown The idea in DeepDream is to choose a layer (or layers) and maximize the "loss" in a way that the image increasingly "excites" the layers. The complexity of the features incorporated depends on layers chosen by you, i.e, lower layers produce strokes or simple patterns, while deeper layers give sophisticated features in images, or even whole objects.

#@markdown The InceptionV3 architecture is quite large (for a graph of the model architecture see TensorFlow's research repo). For DeepDream, the layers of interest are those where the convolutions are concatenated. There are 11 of these layers in InceptionV3, named 'mixed0' though 'mixed10'. Using different layers will result in different dream-like images. Deeper layers respond to higher-level features (such as eyes and faces), while earlier layers respond to simpler features (such as edges, shapes, and textures). Feel free to experiment with the layers selected below, but keep in mind that deeper layers (those with a higher index) will take longer to train on since the gradient computation is deeper.

base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

# Maximize the activations of these layers
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [ ]:
#@title Calculate Loss
#@markdown The loss is the sum of the activations in the chosen layers. The loss is normalized at each layer so the contribution from larger layers does not outweigh smaller layers. Normally, loss is a quantity you wish to minimize via gradient descent. In DeepDream, you will maximize this loss via gradient ascent.

def calc_loss(img, model):
  # Pass forward the image through the model to retrieve the activations.
  # Converts the image into a batch of size 1.
  img_batch = tf.expand_dims(img, axis=0)
  layer_activations = model(img_batch)
  if len(layer_activations) == 1:
    layer_activations = [layer_activations]

  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)

  return  tf.reduce_sum(losses)

In [ ]:
#@title Gradient Ascent
#@markdown Once you have calculated the loss for the chosen layers, all that is left is to calculate the gradients with respect to the image, and add them to the original image.

#@markdown Adding the gradients to the image enhances the patterns seen by the network. At each step, you will have created an image that increasingly excites the activations of certain layers in the network.

#@markdown The method that does this, below, is wrapped in a tf.function for performance. It uses an input_signature to ensure that the function is not retraced for different image sizes or steps/step_size values. See the Concrete functions guide for details.


class DeepDream(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
  )
  def __call__(self, img, steps, step_size):
      print("Tracing")
      loss = tf.constant(0.0)
      for n in tf.range(steps):
        with tf.GradientTape() as tape:
          # This needs gradients relative to `img`
          # `GradientTape` only watches `tf.Variable`s by default
          tape.watch(img)
          loss = calc_loss(img, self.model)

        # Calculate the gradient of the loss with respect to the pixels of the input image.
        gradients = tape.gradient(loss, img)

        # Normalize the gradients.
        gradients /= tf.math.reduce_std(gradients) + 1e-8 
        
        # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
        # You can update the image by directly adding the gradients (because they're the same shape!)
        img = img + gradients*step_size
        img = tf.clip_by_value(img, -1, 1)

      return loss, img

In [ ]:
#@title Main Loop
deepdream = DeepDream(dream_model)

def run_deep_dream_simple(img, steps=100, step_size=0.01):
  # Convert from uint8 to the range expected by the model.
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  img = tf.convert_to_tensor(img)
  step_size = tf.convert_to_tensor(step_size)
  steps_remaining = steps
  step = 0
  while steps_remaining:
    if steps_remaining>100:
      run_steps = tf.constant(100)
    else:
      run_steps = tf.constant(steps_remaining)
    steps_remaining -= run_steps
    step += run_steps

    loss, img = deepdream(img, run_steps, tf.constant(step_size))
    
    display.clear_output(wait=True)
    show(deprocess(img))
    print ("Step {}, loss {}".format(step, loss))


  result = deprocess(img)
  display.clear_output(wait=True)
  show(result)

  return result

In [ ]:
#@title Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Create Deep Dream Video From One Image
#@markdown Using the File directory to the left, create a folder in Google Drive. Copy path to you folder and paste below:

path = '/content/drive/Shareddrives/D.ai TRIP MEDIA/D.ai MODELS/In Development/Deep Dream/test' #@param {type:"string"}

#@markdown Upload your starting image to your folder and rename it 'img_0000.jpg' (It must be .jpg for now)

#@markdown Parameters:
x_size = 600 #@param {type:"number"}
y_size = 600 #@param {type:"number"}
max_images =  300#@param {type:"number"}
OCTAVE_SCALE = 1.10 #@param {type:"number"}
x_trim =  2#@param {type:"number"}
y_trim = 1 #@param {type:"number"}

start = time.time()
created_count = 0
for i in range(0, 9999999999999999):

    if os.path.isfile('{}/img_{:04}.jpg'.format(path, i+1)):
        print('{} already exists, continuing along...'.format(i+1))

    else:
        dream_path = '{}/img_{:04}.jpg'.format(path, i)
        img_result = download(dream_path, max_dim=max(x_size,y_size))

        img_result = img_result[0+x_trim:y_size-y_trim, 0+y_trim:x_size-x_trim]
        img_result = cv2.resize(img_result, (x_size, y_size))

        # Use these to modify the general colors and brightness of results.
        # results tend to get dimmer or brighter over time, so you want to
        # manually adjust this over time.

        # # +2 is slowly dimmer
        # # +3 is slowly brighter
        # img_result[:, :, 0] += 3  # reds
        # img_result[:, :, 1] += 3  # greens
        # img_result[:, :, 2] += 3  # blues

        img_result = np.clip(img_result, 0.0, 255.0)
        img_result = img_result.astype(np.uint8)

        img_result = tf.constant(np.array(img_result))
        base_shape = tf.shape(img_result)[:-1]
        float_base_shape = tf.cast(base_shape, tf.float32)

        for n in range(-2, 3):
          new_shape = tf.cast(float_base_shape*(OCTAVE_SCALE**n), tf.int32)

          img_result = tf.image.resize(img_result, new_shape).numpy()

          img_result = run_deep_dream_simple(img=img_result, steps=5, step_size=0.01)

        display.clear_output(wait=True)
        img_result = tf.image.resize(img_result, base_shape)
        img_result = tf.image.convert_image_dtype(img_result/255.0, dtype=tf.uint8)
        img_result = np.clip(img_result, 0.0, 255.0)
        img_result = img_result.astype(np.uint8)
        result = PIL.Image.fromarray(img_result, mode='RGB')
        result.save('{}/img_{:04}.jpg'.format(path, i+1))

        created_count += 1
        if created_count > max_images:
            break

end = time.time()
end-start

In [ ]:
from tqdm.notebook import tqdm
from PIL import ImageFile, Image

#@markdown **Generate a video with the result (You can edit frame rate and stuff by double-clicking this tab)**
first_frame = 0#@param {type:"number"} #This is the frame where the video will start
last_frame = 300 #@param {type:"number"} #You can change i to the number of the last frame you want to generate. It will raise an error if that number of frames does not exist.

# min_fps = 30#@param {type:"number"}
# max_fps = 30#@param {type:"number"}

# total_frames = last_frame-init_frame

# length = 5#@param {type:"number"} #Desired video time in seconds

frames = []
tqdm.write('Generating video...')
for i in range(first_frame,last_frame): 
    filename = f"{path}/img_{i:04}.jpg"
    frames.append(Image.open(filename))

#fps = last_frame/10
fps = 30#@param {type:"number"}

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'png')
p.stdin.close()

print("The video is now being compressed, wait...")
p.wait()
print("The video is ready")

In [ ]:
#@markdown #**Download the result video**
from google.colab import files
files.download("video.mp4")